# Patch Analyzer

We will use this to gather all patches info

In [1]:
tools = [
    'Elysium', 
    'sGuard', 
    'sGuardPlus',
    'SmartFix', 
    'SmartShield', 
    "SolGPT" ,
    'TIPS', 
]

In [2]:
Columns=['Patch','Original','Category','Tool','DIFF','COMP', 'Valid Patches', 'Detected','Fixed', 'Maintained','link']

In [3]:
import os

#collecting contracts name
dataset='smartbugs'
results_path='../results'
contracts_path=os.path.join(results_path,dataset,'contracts.csv')
storage_path=os.path.join(results_path,dataset)

import pandas as pd
contracts_df = pd.read_csv(contracts_path)

import pandas as pd
result_df = pd.DataFrame([], columns=Columns)

In [4]:
result_df

,Patch,Original,Category,Tool,DIFF,COMP,Valid Patches,Detected,Fixed,Maintained,link


In [5]:
import re
def get_contract(input_string):
    parts = input_string.split('/')
    return parts[-2]  

def get_category(text):
    match = re.search(r'^([^/]+)', text)
    if match:
        return match.group(1)
    else:
        print('fail on get_category')
        return ''
def get_patch(input_string):
    parts = input_string.split('/')
    return parts[-1]

In [6]:
#test

def test_n_patches(result_df):
    assert len(result_df[result_df['Tool'] == 'Elysium'].values.tolist()) == 126
    assert len(result_df[result_df['Tool'] == 'SmartShield'].values.tolist()) == 134
    assert len(result_df[result_df['Tool'] == 'sGuard'].values.tolist()) == 109
    assert len(result_df[result_df['Tool'] == 'sGuardPlus'].values.tolist()) == 81
    assert len(result_df[result_df['Tool'] == 'SmartFix'].values.tolist()) == 86
    assert len(result_df[result_df['Tool'] == 'TIPS'].values.tolist()) == 242
    assert len(result_df[result_df['Tool'] == 'SolGPT'].values.tolist()) == 552
def test_dataset(result_df):
    assert len(result_df['Original'].unique()) == 141
    assert len(result_df['Category'].unique()) ==10
    assert len(result_df['Tool'].unique()) == 7

def test_diff_patches(result_df):
    assert len(result_df[(result_df['Tool'] == 'Elysium') & (result_df['DIFF'] == True)]) == 101
    assert len(result_df[(result_df['Tool'] == 'SmartShield') & (result_df['DIFF'] == True)]) == 131
    assert len(result_df[(result_df['Tool'] == 'sGuard') & (result_df['DIFF'] == True)]) == 62
    assert len(result_df[(result_df['Tool'] == 'sGuardPlus') & (result_df['DIFF'] == True)]) == 81
    assert len(result_df[(result_df['Tool'] == 'SmartFix') & (result_df['DIFF'] == True)]) == 86
    assert len(result_df[(result_df['Tool'] == 'TIPS') & (result_df['DIFF'] == True)]) == 242
    assert len(result_df[(result_df['Tool'] == 'SolGPT') & (result_df['DIFF'] == True)]) == 552
def test_compilable_patches(result_df):
    assert len(result_df[(result_df['Tool'] == 'Elysium') & (result_df['COMP'] == True)]) == 0  # Elysium does not have compilable data
    assert len(result_df[(result_df['Tool'] == 'SmartShield') & (result_df['COMP'] == True)]) == 0  # Smartshield does not have compilable data
    assert len(result_df[(result_df['Tool'] == 'sGuard') & (result_df['COMP'] == True)]) == 108
    assert len(result_df[(result_df['Tool'] == 'sGuardPlus') & (result_df['COMP'] == True)]) == 81
    assert len(result_df[(result_df['Tool'] == 'SmartFix') & (result_df['COMP'] == True)]) == 86
    assert len(result_df[(result_df['Tool'] == 'TIPS') & (result_df['COMP'] == True)]) == 234
    assert len(result_df[(result_df['Tool'] == 'SolGPT') & (result_df['COMP'] == True)]) == 527

def test_valid_patches(result_df):
    assert len(result_df[(result_df['Tool'] == 'Elysium') & (result_df['Valid Patches'] == True)]) == 101  # Elysium does not have compilable data
    assert len(result_df[(result_df['Tool'] == 'SmartShield') & (result_df['Valid Patches'] == True)]) == 131  # Smartshield does not have compilable data
    assert len(result_df[(result_df['Tool'] == 'sGuard') & (result_df['Valid Patches'] == True)]) == 61
    assert len(result_df[(result_df['Tool'] == 'sGuardPlus') & (result_df['Valid Patches'] == True)]) == 81
    assert len(result_df[(result_df['Tool'] == 'SmartFix') & (result_df['Valid Patches'] == True)]) == 86
    assert len(result_df[(result_df['Tool'] == 'TIPS') & (result_df['Valid Patches'] == True)]) == 234
    assert len(result_df[(result_df['Tool'] == 'SolGPT') & (result_df['Valid Patches'] == True)]) == 527




In [7]:
def read_diff(tools, result_df):
    for tool in tools:
        tool_diff_file='../results/smartbugs/'+tool+'/patches_diff.csv'
        df_diff=pd.read_csv(tool_diff_file, names=['Patch', 'Original','DIFF'], skiprows=1)
        if tool in ['SolGPT', 'Elysium']:
            df_diff=pd.read_csv(tool_diff_file, names=['Patch', 'Original','cleaned','DIFF'], skiprows=1)
        values=df_diff.values.tolist()
        for line in values:
            category= get_category(line[0])
            patch = get_patch(line[0])
            original= get_contract(line[0])+ '.sol'
            diff= line[-1]
            link= 'https://github.com/ASSERT-KTH/RepairComp/blob/main/results/smartbugs/'+tool+'/'+line[0]
            new_row = {'Patch': patch, 'Category': category, 'Original': original, 'Tool':tool, 'DIFF': diff,'link':link}
            result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
    return result_df
                   


In [8]:
def read_comp(tools,result_df):
    for tool in tools:
        tool_comp_file='../results/smartbugs/'+tool+'/compilation_results_0.4.24.csv'
        df_comp=pd.read_csv(tool_comp_file, names=['Patch', 'COMP'], skiprows=1)
        values=df_comp.values.tolist()
        for line in values:
            category= get_category(line[0])
            patch = get_patch(line[0])
            original= get_contract(line[0])+ '.sol'
            comp= line[-1]
            if comp == 0:
                comp = True
            else :
                comp = False
            mask = (result_df['Patch'] == patch) & (result_df['Category'] == category) & (result_df['Tool'] == tool)
            if mask.any():
                result_df.loc[mask, 'COMP'] = comp
    return result_df
    

In [9]:
def read_d_f_m(tools,result_df):
    for tool in tools:
        tool_dfm_file='../results/smartbugs/'+tool+'/patches_evaluation.csv'
        df_dfm=pd.read_csv(tool_dfm_file, names=['Patch', 'Original','Detected', 'Fixed','Maintained'], skiprows=1)
        values=df_dfm.values.tolist()
        for line in values:
            category= get_category(line[0])
            patch = get_patch(line[0])
            original= get_contract(line[0])+ '.sol'
            D= line[-3]
            F=line[-2]
            M=line[-1]
            mask = (result_df['Patch'] == patch) & (result_df['Category'] == category) & (result_df['Tool'] == tool)
            result_df.loc[mask, 'Detected'] = D
            result_df.loc[mask, 'Fixed'] = F
            result_df.loc[mask, 'Maintained'] = M
    return result_df

In [10]:
result_df = pd.DataFrame([], columns=Columns)    
result_df=read_diff(tools, result_df)

comp_tools=[
    #'Elysium', 
    'sGuard', 
    'sGuardPlus',
    'SmartFix', 
    #'SmartShield', 
    "SolGPT" ,
    'TIPS', 
]
result_df=read_comp(comp_tools, result_df)

import numpy as np
cond1= result_df['Tool'].isin(comp_tools)&result_df['COMP'] & result_df['DIFF']
cond2= (result_df['Tool'].isin(comp_tools)==False) & result_df['DIFF']
result_df['Valid Patches'] = np.where(cond1 | cond2, True, False)
result_df=read_d_f_m(tools,result_df)

In [11]:
result_df[(result_df['Valid Patches']== True) & (result_df['Fixed'].isin( [0,1,2,3,4,5,6]) == False)]

,Patch,Original,Category,Tool,DIFF,COMP,Valid Patches,Detected,Fixed,Maintained,link
132,lucky_doubler.sol,lucky_doubler.sol,bad_randomness,sGuard,True,True,True,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...
146,0x7a8721a9d64c74da899424c1b52acbf58ddc9782.sol,0x7a8721a9d64c74da899424c1b52acbf58ddc9782.sol,reentrancy,sGuard,True,True,True,2.0,-1.0,3.0,https://github.com/ASSERT-KTH/RepairComp/blob/...
194,0x4a66ad0bca2d700f11e1f2fc2c106f7d3264504c.sol,0x4a66ad0bca2d700f11e1f2fc2c106f7d3264504c.sol,unchecked_low_level_calls,sGuard,True,True,True,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...
224,etherpot_lotto.sol,etherpot_lotto.sol,unchecked_low_level_calls,sGuard,True,True,True,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...


In [12]:
#sanitychecks
test_n_patches(result_df)
test_dataset(result_df)
test_diff_patches(result_df)
test_compilable_patches(result_df)
test_valid_patches(result_df)

In [13]:
result_df

,Patch,Original,Category,Tool,DIFF,COMP,Valid Patches,Detected,Fixed,Maintained,link
0,short_address_example.bin,short_address_example.sol,short_addresses,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1,lottopollo.bin,lottopollo.sol,time_manipulation,Elysium,False,NaN,False,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...
2,timed_crowdsale.bin,timed_crowdsale.sol,time_manipulation,Elysium,False,NaN,False,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...
3,ether_lotto.bin,ether_lotto.sol,time_manipulation,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
4,roulette.bin,roulette.sol,time_manipulation,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
...,...,...,...,...,...,...,...,...,...,...,...
1325,multiowned_vulnerable.sol,multiowned_vulnerable.sol,access_control,TIPS,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1326,mapping_write.sol,mapping_write.sol,access_control,TIPS,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1327,wallet_03_wrong_constructor.sol,wallet_03_wrong_constructor.sol,access_control,TIPS,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1328,mycontract.sol,mycontract.sol,access_control,TIPS,True,True,True,1,1,0,https://github.com/ASSERT-KTH/RepairComp/blob/...


In [14]:
#Patch 	Original 	Category 	Tool 	link 	DIFF 	COMP 	Valid Patches 	Fixed 	Maintained 	Undetected
result_df[result_df['Original']=='0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol']#duplicate case

,Patch,Original,Category,Tool,DIFF,COMP,Valid Patches,Detected,Fixed,Maintained,link
23,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.bin,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,Elysium,True,NaN,True,1,1,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
102,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.bin,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
148,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,sGuard,True,True,True,6.0,0.0,6.0,https://github.com/ASSERT-KTH/RepairComp/blob/...
212,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,sGuard,True,True,True,0.0,0.0,0.0,https://github.com/ASSERT-KTH/RepairComp/blob/...
330,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,SmartFix,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
377,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,SmartFix,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
445,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.bin,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,SmartShield,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
504,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.bin,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,SmartShield,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
712,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839_4ro...,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,SolGPT,True,True,True,1,1,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
713,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839_3ro...,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,unchecked_low_level_calls,SolGPT,True,True,True,1,1,0,https://github.com/ASSERT-KTH/RepairComp/blob/...


In [15]:
def turn_to_per_contract(df,tools,column):
    per_contract=pd.read_csv(contracts_path)
    del per_contract['Occurrences'] 
    del per_contract['Lines'] 
    values=df.values.tolist()
    for tool in tools:
        per_contract[tool]=''
        for line in values:
            line_tool=line[3]
            if line_tool==tool:
                patch=line[0]
                original=line[1]
                category=line[2]
                value=line[list(df.keys()).index(column)]
                find= (per_contract['Name']==original) & (per_contract['Category']==category) 
                if value ==True:
                    per_contract.loc[find, tool] = value
    
    return per_contract
        

In [16]:
def turn_to_per_contract_dfm(df,tools,column):
    per_contract=pd.read_csv(contracts_path)
    del per_contract['Occurrences'] 
    del per_contract['Lines'] 
    values=df.values.tolist()
    for tool in tools:
        per_contract[tool]=''
        for line in values:
            line_tool=line[3]
            if line_tool==tool:
                patch=line[0]
                original=line[1]
                category=line[2]
                value=line[list(df.keys()).index(column)]
                find= (per_contract['Name']==original) & (per_contract['Category']==category) 
                if value>0:
                    bool_value= value>0 #means if fixed,detected, or 
                    per_contract.loc[find, tool] = bool_value
    
    return per_contract

In [17]:
per_c_diff=turn_to_per_contract(result_df,tools,'DIFF')
per_c_diff[per_c_diff==True].count()

Name             0
Category         0
Elysium        101
sGuard          62
sGuardPlus      81
SmartFix        86
SmartShield    131
SolGPT         139
TIPS           140
dtype: int64

In [18]:
per_c_comp=turn_to_per_contract(result_df,tools,'COMP')
per_c_comp[per_c_comp==True].count()

Name             0
Category         0
Elysium          0
sGuard         108
sGuardPlus      81
SmartFix        86
SmartShield      0
SolGPT         138
TIPS           138
dtype: int64

In [19]:
per_c_valid=turn_to_per_contract(result_df,tools,'Valid Patches')
per_c_valid[per_c_valid==True].count()

Name             0
Category         0
Elysium        101
sGuard          61
sGuardPlus      81
SmartFix        86
SmartShield    131
SolGPT         138
TIPS           138
dtype: int64

In [20]:
per_c_d=turn_to_per_contract_dfm(result_df,tools,'Detected')
per_c_d[per_c_d==True].count()

Name            0
Category        0
Elysium        53
sGuard         35
sGuardPlus     70
SmartFix       51
SmartShield    60
SolGPT         97
TIPS           82
dtype: int64

In [21]:
per_c_f=turn_to_per_contract_dfm(result_df,tools,'Fixed')
per_c_f[per_c_f==True].count()

Name            0
Category        0
Elysium        53
sGuard          3
sGuardPlus     70
SmartFix       50
SmartShield    40
SolGPT         89
TIPS           81
dtype: int64

In [22]:
result_df

,Patch,Original,Category,Tool,DIFF,COMP,Valid Patches,Detected,Fixed,Maintained,link
0,short_address_example.bin,short_address_example.sol,short_addresses,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1,lottopollo.bin,lottopollo.sol,time_manipulation,Elysium,False,NaN,False,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...
2,timed_crowdsale.bin,timed_crowdsale.sol,time_manipulation,Elysium,False,NaN,False,NaN,NaN,NaN,https://github.com/ASSERT-KTH/RepairComp/blob/...
3,ether_lotto.bin,ether_lotto.sol,time_manipulation,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
4,roulette.bin,roulette.sol,time_manipulation,Elysium,True,NaN,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
...,...,...,...,...,...,...,...,...,...,...,...
1325,multiowned_vulnerable.sol,multiowned_vulnerable.sol,access_control,TIPS,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1326,mapping_write.sol,mapping_write.sol,access_control,TIPS,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1327,wallet_03_wrong_constructor.sol,wallet_03_wrong_constructor.sol,access_control,TIPS,True,True,True,0,0,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
1328,mycontract.sol,mycontract.sol,access_control,TIPS,True,True,True,1,1,0,https://github.com/ASSERT-KTH/RepairComp/blob/...
